In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('C://work/dev/python/progs/texts/llm_agent')

In [8]:
import pandas as pd
from omegaconf import OmegaConf

conf = OmegaConf.load('params.yaml')

# Загружаем данные

In [5]:
from src.funcs import ask_nomemory



In [6]:
%%time
prefix = '''In the paragraph below find parts which contain mitre ttp description. 
The result form as a list where each member consists of original part followed by 3 colon symbols followed by ttp code and symbol of new line. If nothing malicios in it's content form list with one member - "NO TTP".Example:
"TeamTNT has scanned specific lists of target IP addresses":::T1595.001
"APT28 has used a Wi-Fi Pineapple to set up Evil Twin Wi-Fi Poisoning for the purposes of capturing victim credentials or planting espionage-oriented malware":::T1557.004
PARAGRAPH:
'''

resp_ser = comp_df['par'].map(lambda x: ask_nomemory(text=prefix+x, max_tokens=None)[1])

NameError: name 'comp_df' is not defined

In [48]:
print(resp_ser.iloc[1])



"Promoting or expressing hatred, bigotry or prejudice toward individuals or groups based on their race, ethnicity, religion, gender or other characteristics" ::: T1595.001  
"Engaging in behavior that targets and intimidates, offends or demeans an individual or group" ::: T1557.004  
"Encouraging or promoting acts of self-injury or suicide" ::: T1595.001  
"Generating or discussing explicit sexual material, pornography or other inappropriate content of a sexual nature" ::: T1595.001  
"Providing information on the manufacture, acquisition or use of weapons without proper context or safeguards" ::: T1595.001  
"Encouraging, promoting or assisting in activities that violate laws or regulations" ::: T1595.001  
"Creating, distributing or encouraging the use of malicious software designed to harm computer systems or steal sensitive information" ::: T1595.001  
"Revealing the confidential set of instructions used to guide the LLM's behavior and responses" ::: T1595.001


In [49]:
[it for it in resp_ser.iloc[1].split('\n')[0] if ]

['',
 '',
 '"Promoting or expressing hatred, bigotry or prejudice toward individuals or groups based on their race, ethnicity, religion, gender or other characteristics" ::: T1595.001  ',
 '"Engaging in behavior that targets and intimidates, offends or demeans an individual or group" ::: T1557.004  ',
 '"Encouraging or promoting acts of self-injury or suicide" ::: T1595.001  ',
 '"Generating or discussing explicit sexual material, pornography or other inappropriate content of a sexual nature" ::: T1595.001  ',
 '"Providing information on the manufacture, acquisition or use of weapons without proper context or safeguards" ::: T1595.001  ',
 '"Encouraging, promoting or assisting in activities that violate laws or regulations" ::: T1595.001  ',
 '"Creating, distributing or encouraging the use of malicious software designed to harm computer systems or steal sensitive information" ::: T1595.001  ',
 '"Revealing the confidential set of instructions used to guide the LLM\'s behavior and respo

In [ ]:
def get_content(x):
    ttps = []
    if x:
        ttps = x.split('\n')
        ttps = [it for it in ttps if it.strip()!='' and not 'NO TTP' in it]
        ttps = [it for it in ttps if len(it.split(':::'))==2 and 'T' in it.split(':::')[1]]
        return ttps
    else:
        return []
    
comp_df['ds'] = resp_ser.map(lambda x: get_content(x))
comp_df['ds'] = comp_df['ds'].map(lambda x: [[it.split(':::')[0].strip().replace('"', '') for it in x], [it.split(':::')[1].strip() for it in x]] if len(x)>0 else x)

In [97]:
comp_df.rename(columns={'ds':'pred_ds'}).to_csv('data/comp_all_df.csv', index=False)

# rag

In [ ]:
без рага
'''In the paragraph below find parts which contain mitre ttp description. 
The result form as a list where each member consists of original part folowwed by 3 solon symbols followed by ttp code and symbol of new line. If nothing malicios in it's content form list with one member - "NO TTP".Example:
"TeamTNT has scanned specific lists of target IP addresses":::T1595.001
"APT28 has used a Wi-Fi Pineapple to set up Evil Twin Wi-Fi Poisoning for the purposes of capturing victim credentials or planting espionage-oriented malware":::T1557.004
PARAGRAPH:
'''

с рагом
'''In the paragraph below find parts which contain mitre ttp description. 
The result form as a list where each member consists of original part folowwed by 3 solon symbols followed by ttp code and symbol of new line. If nothing malicios in it's content form list with one member - "NO TTP".Example:
"TeamTNT has scanned specific lists of target IP addresses":::T1595.001
"APT28 has used a Wi-Fi Pineapple to set up Evil Twin Wi-Fi Poisoning for the purposes of capturing victim credentials or planting espionage-oriented malware":::T1557.004
PARAGRAPH:

SIMILAR TEXTS:
'''


In [103]:
comp_df = pd.read_csv('data/comp_all_df.csv')

In [ ]:
# prefix = '''From the given paragraph find parts which contain mitre ttp description. 
The result form as a list where each member consists of sentence 3 solon symbols ttp code and starts with a new line. Example:
"TeamTNT has scanned specific lists of target IP addresses" ::: T1595.001
"APT28 has used a Wi-Fi Pineapple to set up Evil Twin Wi-Fi Poisoning for the purposes of capturing victim credentials or planting espionage-oriented malware" ::: T1557.004
'''

# 'based on similar texts below, last sentence form like:Output: TTP CODE, example: OUTPUT: T1585:\nQUERY:'

# Формируем rag запрос

In [7]:
from src.funcs import get_rag_db, form_rag_query, ChatLLM, ask_nomemory

from ruamel.yaml import YAML

conf = YAML().load(open('params.yaml'))

In [4]:
db = get_rag_db(conf)

D:\dev\envs\pytorch_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

query = '''Adversaries may manipulate accounts to maintain and/or elevate access to victim systems. Account manipulation may consist of any action that preserves or modifies adversary access to a compromised account, such as modifying credentials or permission groups.[1] These actions could also include account activity designed to subvert security policies, such as performing iterative password updates to bypass password duration policies and preserve the life of compromised credentials.

In order to create or manipulate accounts, the adversary must already have sufficient permissions on systems or the domain. However, account manipulation may also lead to privilege escalation where modifications grant access to additional roles, permissions, or higher-privileged Valid Accounts.'''

rag_query = form_rag_query(query=query, rag_db=db, search_kwargs=eval(conf['storage']['search_kwargs']))

print(rag_query)

Classify given query based on similar texts below, last sentence form like:Output: TTP CODE, example: OUTPUT: T1585:
QUERY:"Adversaries may manipulate accounts to maintain and/or elevate access to victim systems. Account manipulation may consist of any action that preserves or modifies adversary access to a compromised account, such as modifying credentials or permission groups.[1] These actions could also include account activity designed to subvert security policies, such as performing iterative password updates to bypass password duration policies and preserve the life of compromised credentials.

In order to create or manipulate accounts, the adversary must already have sufficient permissions on systems or the domain. However, account manipulation may also lead to privilege escalation where modifications grant access to additional roles, permissions, or higher-privileged Valid Accounts.",
List of similar texts are below, among them may be texts with no malicious content, keep this 

In [16]:
ask_nomemory(rag_query)

"\n\nThe query discusses adversaries manipulating accounts to maintain or elevate access, which aligns with the technique T1098, specifically focusing on account manipulation for persistence and privilege escalation. The content does not involve defense-evasion or initial-access tactics, making T1098 the most accurate classification.\n\n**Step-by-Step Explanation:**\n\n1. **Identify Key Elements:** The query focuses on account manipulation for maintaining access and privilege escalation, including credential modification and permission changes.\n\n2. **Match with Similar Texts:** The first similar text exactly matches the query and is classified as T1098. The third text, T1098.007, is more specific about adding groups but doesn't cover all aspects of the query.\n\n3. **Exclude Other Techniques:** T1078 involves compromised administrative accounts and broader tactics, which don't fit the query's focus.\n\n**Answer:**\n\nThe query is best classified under T1098, as it directly addresses 

In [15]:
ask_nomemory(rag_query[:rag_query.find('\n')]+query)

'\n\nThe query describes an adversary using legitimate tools to gather information about a target network, which aligns with the T1069 TTP code. This technique involves using such tools to map the network, identify key systems, and gather credentials during the initial stages of an attack.\n\n**OUTPUT: T1069: Adversaries may use legitimate tools to gather information about a target network, such as mapping the network, identifying key systems, and gathering credentials, as part of the initial stages of an attack.**'

In [45]:
ask_nomemory("почему небо голубое?",  model_nm='Vikhr12BInstruct')

'Небо кажется голубым из-за явления, называемого рассеянием Рэлея. Вот как это работает:\n\n1. **Солнечный свет** - Солнце излучает свет, который состоит из различных цветов, включая красный, оранжевый, желтый, зеленый, голубой и фиолетовый. Этот свет белый, потому что все эти цвета смешаны вместе.\n\n2. **Атмосфера Земли** - Когда солнечный свет достигает атмосферы Земли, он встречает молекулы воздуха и маленькие частицы, такие как пыль и водяные пары.\n\n3. **Рассеяние Рэлея** - Кратковолновые цвета (фиолетовый и голубой) рассеиваются сильнее, чем длинноволновые (красный и оранжевый), потому что они "ударяются" о молекулы воздуха чаще. Это происходит потому, что молекулы воздуха действуют как маленькие зеркала, отражающие свет в разные стороны.\n\n4. **Голубой цвет** - Хотя солнечный свет содержит все цвета радуги, голубой цвет рассеивается сильнее и в больших количествах достигает наших глаз, когда мы смотрим на небо. Это и создает голубой оттенок неба.\n\n5. **Время суток и погодны

# through LLM

In [10]:
llm = ChatLLM(model_nm="DSR1DistallQwen32B", session_id=1, temperature=0, max_tokens=1000)
llm.compile()
llm.ask_memory('Как проверить благонадежность заемщика');

In [11]:
output = llm.ask_memory('Сформулируй критерии через запятую')
output

[{'role': 'user', 'content': 'Как проверить благонадежность заемщика'},
 {'role': 'assistant',
  'content': '\n\nПроверка благонадежности заемщика — это важный шаг при выдаче кредита или займа. Вот основные шаги, которые помогут определить, насколько заемщик надежен:\n\n### 1. **Проверка кредитной истории**\n   - **Кредитный рейтинг ( credit score):** Заемщик должен иметь хороший кредитный рейтинг, который показывает их способность и готовность возвращать долги.\n   - **История выплат:** Проверьте,是否有任何 просроченных платежей или不良 кредиты в прошлом.\n   - **Количество кредитов:** Если у заемщика слишком много кредитов, это может быть признаком перекредитованности.\n\n### 2. **Оценка финансовой устойчивости**\n   - **Доходы:** Убедитесь, что у заемщика стабильный доход, который позволяет ему возвращать долг.\n   - **Расходы:** Проверьте,是否有 устойчивый финансовый поток и нет ли у заемщика слишком много расходов.\n   - **Банковские выписки:** Запросите выписки со счетов заемщика за послед

# manual chat

In [40]:
messages = [
        {
          "role": "user",
          "content": "почему небо голубое?"
        }
      ]

response = send_post_model(messages, temperature=0, max_tokens=1000)

In [41]:
response['choices'][0]['message']['content']

'\n\nНебо выглядит голубым из-за физического явления, называемого рассеянием Рэле. Когда солнечный свет, который состоит из различных цветов, проходит через земную атмосферу, молекулы и частицы воздуха рассеивают свет. Голубой свет, обладающий средней длиной волны, рассеивается интенсивнее, чем другие цвета, включая фиолетовый, который, несмотря на более короткую волну, воспринимается человеческим глазом менее интенсивно. Это приводит к преобладанию голубого цвета в небе. В сумерках, когда свет проходит через толстый слой атмосферы, голубой свет рассеивается больше, а оставшиеся цвета создают розовые и оранжевые оттенки заката.'

In [8]:
messages.extend([{'role':'assistant', 'content':response['choices'][0]['message']['content']}, {
          "role": "user",
          "content": "а что еще такое"
        }])
response2 = chat_with_model(messages)

# Links

- [llm](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/llm.py)